#IMPORTING

In [1]:
import random
import pandas as pd

In [2]:
data_2022 = pd.read_excel('/kaggle/input/digital-and-data-leaders-survey/data_2022.xlsx')
pd.DataFrame(data_2022)

data_2020 = pd.read_excel('/kaggle/input/digital-and-data-leaders-survey/data_2020.xlsx')
pd.DataFrame(data_2020)

,Date Started,Industry,Role,How many data specialists does your organisation have?*,Your organisation has sponsorship or a champion at the exective level for data and analytics.*,Your organisation has trust in the data and analytics available to them.,Your organisation uses data and insights to inform its corporate strategy.*,Your organisation has an enterprise-wide data and analytics strategy being used.,Your organisation makes automated data-driven decisions*,Which structure most closely resembles your organisation?*,...,Which technologies does your organisation use? (choose multiple)*,Which data and analytics roles does your organisation have? (choose multiple)*,Which of these analytical techniques does your organisation use as part of a business process? (choose multiple)*,Does your organisation use cloud for data and analytics? (choose multiple)*,Which methodologies do you use when delivering data and analytics projects? (choose multiple)*,How are the majority of analytical insights delivered in your organisation?*,Your organisation outsources the development of data and/or analytics.*,"Do you think New Zealand is different from other countries in its use of data and analytics? If so, why?",What are the biggest challenges in getting your organisation to use data and analytics to make decisions?,What do you think your organisation should be doing more of in relation to data?
0,2020-08-27,"Transport, Postal and Warehousing",Director of Analytics,6-20,Agree,Agree,Strongly agree,Both strategy and implementation are strong,Some decisions are made automatically with ana...,Matrix - analytics specialists are in cross-fu...,...,Machine learning pipeline; Visualisation tools...,Analysts; Data scientists; Data engineers; BI ...,Advanced statistics; Predictive modelling,AWS; Microsoft Azure,Waterfall; Agile; Design thinking,markdown/shiny/,None,yes in terms of the pragmatism and low adoptio...,understanding the complexity involved in each ...,share a common understanding of what is requir...
1,2020-08-27,Information Media and Telecommunications,"General Manager, Data and Insights",6-20,Strongly agree,Strongly agree,Strongly agree,We have a strategy with partial implementation,Some decisions are made automatically with ana...,Centralised - analytics specialists sit in a c...,...,"Visualisation tools (e.g. Power BI, Tableau, Q...",Analysts; Data scientists; Data engineers; BI ...,Advanced statistics; Predictive modelling; Mac...,AWS; Microsoft Azure; Google,Agile,Analysis/insights are distributed in Excel,Some,I think NZ are still relatively early on the a...,The business is very supportive of using data ...,We have spent the last few years getting our d...
2,2020-08-27,Information Media and Telecommunications,Chpater Lead - Data Science,50+,Strongly agree,Agree,Agree,Both strategy and implementation are strong,Some decisions are made automatically with ana...,Federated - analytics specialists are both emb...,...,Machine learning pipeline; Visualisation tools...,Data storytellers; Analysts; Data scientists; ...,Predictive modelling; Machine learning or AI,Microsoft Azure,Agile; DevOps,Interactive dashboards are used,None,Yes. Because of the scale there is less intere...,Buy in from senior stakeholders,More use cases
3,2020-08-28,Public Administration and Safety,"Strategic Leader - Customer strategy ,Data and...",50+,Strongly agree,Strongly agree,Agree,We have a strategy with partial implementation,Some decisions are made automatically with ana...,Federated - analytics specialists are both emb...,...,"Visualisation tools (e.g. Power BI, Tableau, Q...",Analysts; Data scientists; Data engineers; Dat...,Advanced statistics; Predictive modelling,AWS,Agile; DataOps,Analytics are operationalised as part of a bus...,Some,"Yes, it is behind in many ways. Lack of approp...",My organisation is well advanced in its thinki...,"Our org is well advanced, change and time is w..."
4,2020-08-28,"Professional, Scientific and Technical Services",Digital 

#PROCESSING


###In order to gain comparable insights:
- we need to find common questions in both datasets
- Then clean both of the columns to match the value type
- Then run the analysis & visualisations

###What insights do I aim to find out for 2022 Analysis 

First I will Segregate into industries:
- Gauging the scale of digital transformation across various industries
- Learning about the sentiment of the people towards digital transformation, measuring what kind of feedback is the most important according to them.
- What's the correlation between one industry and another (ripple effects of transformation?)

##2020 vs 2022
- What was the general growth over the years in the industry.

In [3]:
data_2022 = data_2022.drop(['ID', 'Start time',	'Completion time'],axis = 1)
data_2020 = data_2020.drop(['Date Started'],axis = 1)

# Seggregating based on industries, each industry name has a table associated with it!

In [4]:
# Finding unique industry names -> x
for i in data_2022['Which industry do you primarily operate in?']:
  data_2022['Which industry do you primarily operate in?'] = data_2022['Which industry do you primarily operate in?'].replace([i], "".join(i.split()))

industries = [] 
for i in data_2022['Which industry do you primarily operate in?']:
  industries.append(i)

set(industries) #Now type any name from here to get the indsutrial values

# Making different tables for different industries (per industry data is low, needed for comparison of growth)

for i in industries:
  #converting string to variable
  str = i
  locals()[str] = data_2022.loc[data_2022['Which industry do you primarily operate in?'] == i]

In [5]:
list(data_2022.columns)

['Which industry do you primarily operate in?',
 'Which field is the best fit for your current position?',
 'My organisation acquires and stores data from external sources.',
 'My organisation has a centralized data repository.',
 'My organisation makes automated data-driven decisions.',
 'My organisation has a need to collect or manage internal or external ESG data.',
 'My organisation has trust in the data and analytics available to them.',
 'My organisation has an enterprise-wide data and analytics strategy being used.',
 'My organisation uses data and insights to inform its corporate strategy.',
 'My organisation has sponsorship or a champion at the executive level for data and analytics.',
 'My organisation has people responsible and empowered to improve data quality.',
 'It is easy for everyone in my organisation to access data that they are allowed to view.',
 'How many data specialists does your organisation have?',
 'Which structure most closely resembles your organisation?',


# Question 12, 13 pose a challenge. I will combine these column values together into a single column

Question 12 Processing [DONE]
- Replacing Don't know and Never with No while other values are replaces with the name of technology so that we can combine the tech stack together later by combining the columns
- Replace ; with ,
- Remove Never, Don't know with No
- Combine all the columns together
- Once all combined we get big values, remove repetitive 'No'

# Cloud Tech used by company

In [6]:
Technologies = ['Amazon Web Services','Google Cloud Platform','Microsoft Azure', 'Oracle Cloud Infrastructure ','IBM Cloud']

Answers = list(data_2022['Amazon Web Services'].unique())
Answers.remove("Don't Know")
Answers.remove('Never')

for i in Technologies:
  data_2022[i] = data_2022[i].replace(to_replace='Never', value='No')
  data_2022[i] = data_2022[i].replace(to_replace="Don't Know", value='No')

  for j in data_2022[i]:
    if j in Answers:
      data_2022[i] = data_2022[i].replace(to_replace= j, value= i)

data_2022['Amazon Web Services'].unique()

data_2022['Cloud Technology'] = data_2022[['Amazon Web Services','Google Cloud Platform','Microsoft Azure', 
                                           'Oracle Cloud Infrastructure ','IBM Cloud']].agg(', '.join, axis=1) # .agg combines the strings together

# Data Tools used by company

In [7]:
Questions = ['Data Governance (Select all that are used)', 'Data Visualisation (Select all that are used)','Modern Data Platform (Select all that are used)',
             'Data Science and ML (Select all that are used)','Data Integration (Select all that are used)', 'Low-Code Tools (Select all that are used)',
             'Robotic Process Automation (Select all that are used)']

for i in Questions:
  data_2022[i] = data_2022[i].replace(to_replace=';', value=',', regex=True) # Replace is done on full string, Regex does it to each character too
  data_2022[i] = data_2022[i].replace(to_replace="Don't Know,", value='No', regex=True)
  data_2022[i] = data_2022[i].replace(to_replace='None,', value='No', regex=True)
   
  for j in data_2022[i]:
    x = j.split(',')
    for q in x:
      if q == '':
        x.remove(q)
        x = ', '.join(x)
        data_2022[i].replace(to_replace= j, value = x, inplace=True )

data_2022['Data Technology'] = data_2022[Questions].agg(', '.join, axis=1)

# Professionals required by a company (Roles)

In [8]:
Roles = ['Data Analyst', 'Data Engineer ', 'Data Scientist', 'UI/UX Designer/Engineer', 'Cloud Engineer', 'DevOps Engineer', 'Software Engineer',
         'Business Analyst', 'Solution Architect', 'Data Governance Specialist','Data Storyteller']
  
requirement = list(data_2022['Data Analyst'].unique())
requirement.remove("Don't Know")
requirement.remove('Not Required')

for i in Roles:
  data_2022[i] = data_2022[i].replace(to_replace= "Don't Know", value='No')
  data_2022[i] = data_2022[i].replace(to_replace= 'Not Required', value='No')
  for j in data_2022[i]:
    if j in requirement:
      data_2022[i] = data_2022[i].replace(to_replace = j, value= i)

data_2022['Which data and analytics roles does your organisation have? (choose multiple)'] = data_2022[Roles].agg(', '.join, axis=1)

data_2022['Which data and analytics roles does your organisation have? (choose multiple)']

0     Data Analyst, Data Engineer , No, No, Cloud En...
1     Data Analyst, Data Engineer , Data Scientist, ...
2     Data Analyst, Data Engineer , Data Scientist, ...
3     Data Analyst, Data Engineer , Data Scientist, ...
4     Data Analyst, Data Engineer , Data Scientist, ...
                            ...                        
67    Data Analyst, Data Engineer , Data Scientist, ...
68    Data Analyst, Data Engineer , Data Scientist, ...
69    No, No, No, No, No, No, No, Business Analyst, ...
70    Data Analyst, Data Engineer , Data Scientist, ...
71    Data Analyst, Data Engineer , Data Scientist, ...
Name: Which data and analytics roles does your organisation have? (choose multiple), Length: 72, dtype: object

# Final Cleaning on all these columns and combining Cloud and Data into one



In [9]:
tech = ['Cloud Technology','Data Technology'] # Combining tech columns
data_2022['Which technologies does your organisation use?'] = data_2022[tech].agg(', '.join, axis=1)
data_2022[['Which technologies does your organisation use?', 'Which data and analytics roles does your organisation have? (choose multiple)']]
Final_Columns = data_2022[['Which technologies does your organisation use?', 'Which data and analytics roles does your organisation have? (choose multiple)']]

# Final cleaned columns for both 2020 and 2022

In [10]:
data_2022['Which technologies does your organisation use?'] = Final_Columns['Which technologies does your organisation use?']
data_2022['Which data and analytics roles does your organisation have? (choose multiple)'] = Final_Columns['Which data and analytics roles does your organisation have? (choose multiple)']

In [11]:
compare_2020 =          ['Industry', 
                         'How many data specialists does your organisation have?*',
                         'Your organisation has trust in the data and analytics available to them.', 
                         'Your organisation uses data and insights to inform its corporate strategy.*',
                         'Your organisation has an enterprise-wide data and analytics strategy being used.', 
                         'Your organisation makes automated data-driven decisions*', 
                         'Which structure most closely resembles your organisation?*', 
                         'Your organisation has people responsible and empowered to improve data quality.', 
                         'How difficult is it for people in your organisation to access data they are allowed to view?*', 
                         'Which technologies does your organisation use? (choose multiple)*', 
                         'Which data and analytics roles does your organisation have? (choose multiple)*', 
                         'Does your organisation use cloud for data and analytics? (choose multiple)*', 
                         'Your organisation outsources the development of data and/or analytics.*',
                         'Year']

compare_2022 =          ['Which industry do you primarily operate in?', 
                         'How many data specialists does your organisation have?', 
                         'My organisation has trust in the data and analytics available to them.',                         
                         'My organisation uses data and insights to inform its corporate strategy.',
                         'My organisation has an enterprise-wide data and analytics strategy being used.',
                         'My organisation makes automated data-driven decisions.',
                         'Which structure most closely resembles your organisation?',
                         'My organisation has people responsible and empowered to improve data quality.',
                         'It is easy for everyone in my organisation to access data that they are allowed to view.',
                         'Which technologies does your organisation use?',
                         'Which data and analytics roles does your organisation have? (choose multiple)',
                         'My organisation uses cloud-based tools for most digital solutions.',
                         'My organisation outsources the development of digital solutions.',
                         'Year']

Adding Year Column

In [12]:
data_2022.head(1)
Year = []
for i in range(len(data_2022)):
  Year.append(2022)
data_2022['Year'] = Year

# Final 2022 Data

In [13]:
Final_2022 = data_2022[compare_2022]
Final_2022

,Which industry do you primarily operate in?,How many data specialists does your organisation have?,My organisation has trust in the data and analytics available to them.,My organisation uses data and insights to inform its corporate strategy.,My organisation has an enterprise-wide data and analytics strategy being used.,My organisation makes automated data-driven decisions.,Which structure most closely resembles your organisation?,My organisation has people responsible and empowered to improve data quality.,It is easy for everyone in my organisation to access data that they are allowed to view.,Which technologies does your organisation use?,Which data and analytics roles does your organisation have? (choose multiple),My organisation uses cloud-based tools for most digital solutions.,My organisation outsources the development of digital solutions.,Year
0,Construction,0,Neutral,Disagree,Disagree,Disagree,Don't Know,Agree,Disagree,"Amazon Web Services, Google Cloud Platform, Mi...","Data Analyst, Data Engineer , No, No, Cloud En...",Strongly Disagree,Neutral,2022
1,InsuranceServices,11 - 20,Agree,Agree,Neutral,Disagree,Federated - data specialists are both embedded...,Agree,Disagree,"Amazon Web Services, Google Cloud Platform, Mi...","Data Analyst, Data Engineer , Data Scientist, ...",Agree,Neutral,2022
2,InsuranceServices,6 - 10,Agree,Agree,Disagree,Agree,Federated - data specialists are both embedded...,Agree,Neutral,"Amazon Web Services, Google Cloud Platform, Mi...","Data Analyst, Data Engineer , Data Scientist, ...",Agree,Neutral,2022
3,Government(Central),50+,Disagree,Disagree,Disagree,Disagree,Federated - data specialists are both embedded...,Agree,Strongly Disagree,"Amazon Web Services, No, Microsoft Azure, No, ...","Data Analyst, Data Engineer , Data Scientist, ...",Disagree,Agree,2022
4,Technology,50+,Strongly Agree,Neutral,Strongly Agree,Disagree,Federated - data specialists are both embedded...,Neutral,Disagree,"Amazon Web Services, Google Cloud Platform, Mi...","Data Analyst, Data Engineer , Data Scientist, ...",Agree,Strongly Disagree,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Government(Local),11 - 20,Agree,Agree,Agree,Neutral,Federated - data specialists are both embedded...,Agree,Agree,"Amazon Web Services, No, Microsoft Azure, No, ...","Data Analyst, Data Engineer , Data Scientist, ...",Agree,Agree,2022
68,"Transport,PostalandWarehousing",50+,Agree,Neutral,Neutral,Neutral,Distributed - data specialists are embedded wi...,Neutral,Neutral,"Amazon Web Services, Google Cloud Platform, Mi...","Data Analyst, Data Engineer , Data Scientist, ...",Neutral,Neutral,2022
69,RetailTrade,0,Agree,Agree,Neutral,Neutral,Don't Know,Agree,Neutral,"Amazon Web Services, Google Cloud Platform, No...","No, No, No, No, No, No, No, Business Analyst, ...",Agree,Agree,2022
70,Government(Local),50+,Disagree,Agree,Disagree,Neutral,Distributed - data specialists are embedded wi...,Neutral,Neutral,"Amazon Web Services, No, Microsoft Azure, No, ...","Data Analyst, Data Engineer , Data Scientist, ...",Agree,Disagree,2022


# Processing 2020 Data

- Replace null, NA, None, ;, values
- Making 2020 data comparable to 2022 Data by aligning both their values

In [14]:
Data2020 = data_2020

In [15]:
Answers = ['[No Answer Entered]', 'N/A', 'NA', 'None']
for i in Data2020.columns:
  Data2020[i] = Data2020[i].replace(to_replace=';', value=',', regex=True) # Replace is done on…', regex=True)


'''
  for j in data_2022[i]:
    x = j.split(',')
    for q in x:
      if q == '':
        x.remove(q)
        x = ', '.join(x)
        data_2022[i].replace(to_replace= j, value = x, inplace=True )
        data_2022['Data Technology'] = data_2022[Questions].agg(', '.join, axis=1)
'''

Year = []
for i in range(len(Data2020)):
  Year.append(2020)
Data2020['Year'] = Year

Final_2020 = Data2020[compare_2020]

In [16]:
Final_2020.head(1)

,Industry,How many data specialists does your organisation have?*,Your organisation has trust in the data and analytics available to them.,Your organisation uses data and insights to inform its corporate strategy.*,Your organisation has an enterprise-wide data and analytics strategy being used.,Your organisation makes automated data-driven decisions*,Which structure most closely resembles your organisation?*,Your organisation has people responsible and empowered to improve data quality.,How difficult is it for people in your organisation to access data they are allowed to view?*,Which technologies does your organisation use? (choose multiple)*,Which data and analytics roles does your organisation have? (choose multiple)*,Does your organisation use cloud for data and analytics? (choose multiple)*,Your organisation outsources the development of data and/or analytics.*,Year
0,"Transport, Postal and Warehousing",6-20,Agree,Strongly agree,Both strategy and implementation are strong,Some decisions are made automatically with ana...,Matrix - analytics specialists are in cross-fu...,There are people responsible and empowered to ...,Easy,"Machine learning pipeline, Visualisation tools...","Analysts, Data scientists, Data engineers, BI ...","AWS, Microsoft Azure",None,2020


In [17]:
Final_2022.head(1)

,Which industry do you primarily operate in?,How many data specialists does your organisation have?,My organisation has trust in the data and analytics available to them.,My organisation uses data and insights to inform its corporate strategy.,My organisation has an enterprise-wide data and analytics strategy being used.,My organisation makes automated data-driven decisions.,Which structure most closely resembles your organisation?,My organisation has people responsible and empowered to improve data quality.,It is easy for everyone in my organisation to access data that they are allowed to view.,Which technologies does your organisation use?,Which data and analytics roles does your organisation have? (choose multiple),My organisation uses cloud-based tools for most digital solutions.,My organisation outsources the development of digital solutions.,Year
0,Construction,0,Neutral,Disagree,Disagree,Disagree,Don't Know,Agree,Disagree,"Amazon Web Services, Google Cloud Platform, Mi...","Data Analyst, Data Engineer , No, No, Cloud En...",Strongly Disagree,Neutral,2022


`First combine and then make the values comparable `

In [18]:
Questions = ['Industry','How many data specialists does your organization have?',
'My organization has people responsible and empowered to improve data quality.',
'My organization has trust in the data and analytics available to them.',
'My organization uses data and insights to inform its corporate strategy.',
'My organization has an enterprise-wide data and analytics strategy being used.',
'My organization makes automated data-driven decisions.',
'Which structure most closely resembles your organization?*',
'It is easy for everyone in my organization to access data that they are allowed to view.',
'Which technologies does your organization use?',
'Which data and analytics roles does your organization have? (choose multiple)',
'My organization uses cloud-based tools for most digital solutions.',
'My organization outsources the development of digital solutions.', 'Year']

Final_2020.columns = [Questions]
Final_2022.columns = [Questions]
Combined_Data = Final_2022.append(Final_2020)
Combined_Data.reset_index(inplace=True, drop=True)

print(len(Final_2020), len(Final_2022), len(Combined_Data))

94 72 166


# Processing for each column

In [19]:
Combined_Data
Combined_Data.columns = Combined_Data.columns.map(''.join) # Converting MultiIndex to Index

## 1st Column

In [20]:
old = Combined_Data['How many data specialists does your organization have?']

old = old.replace(to_replace = "Don't know", value= "Don't Know", regex=True) # Replace is done on…', regex=True)
old = old.replace(to_replace = "None", value= '0', regex=True) # Replace is done on…', regex=True)

for i in range(len(old)):
  if old[i] != "Don't Know":
    old[i] = old[i].replace(' ', '')

Combined_Data['How many data specialists does your organization have?'] = old
Combined_Data['How many data specialists does your organization have?'].unique()

array(['0', '11-20', '6-10', '50+', '1-5', "Don't Know", '21-50', '6-20'],
      dtype=object)

## 2nd Column (Ig this part can be done in the visualisation tools as well)

# Cleaning some more 

In [21]:
df = pd.read_csv('/kaggle/input/digital-and-data-leaders-survey/2020_2022_Data.csv')
df = df.drop(['Unnamed: 0'],axis = 1)

for i in df.columns:

  print('Column: ',i, '| Unique Values = ', df[i].nunique())

Column:  Industry | Unique Values =  38
Column:  How many data specialists does your organization have? | Unique Values =  8
Column:  My organization has people responsible and empowered to improve data quality. | Unique Values =  7
Column:  My organization has trust in the data and analytics available to them. | Unique Values =  7
Column:  My organization uses data and insights to inform its corporate strategy. | Unique Values =  10
Column:  My organization has an enterprise-wide data and analytics strategy being used. | Unique Values =  8
Column:  My organization makes automated data-driven decisions. | Unique Values =  10
Column:  Which structure most closely resembles your organization?* | Unique Values =  9
Column:  It is easy for everyone in my organization to access data that they are allowed to view. | Unique Values =  10
Column:  Which technologies does your organization use? | Unique Values =  107
Column:  Which data and analytics roles does your organization have? (choose mu

In [22]:
for i in df.columns:
  df[i] = df[i].replace(to_replace= "Strongly agree", value='Strongly Agree', regex = True)
  df[i] = df[i].replace(to_replace= 'Strongly disagree', value='Strongly Disagree', regex = True)
  df[i] = df[i].replace(to_replace= 'Strongly Agree ', value='Strongly Agree', regex = True)
  df[i] = df[i].replace(to_replace= 'Neutral', value="Don't Know", regex = True)
  df[i] = df[i].replace(to_replace= '[No Answer Entered]', value = "Don't Know")

# ----------

temp = df['My organization uses data and insights to inform its corporate strategy.']

temp = temp.replace(to_replace= "Both strategy and implementation are strong", value = 'Strongly Agree', regex = True)
temp = temp.replace(to_replace= "We have a strategy with partial implementation", value = 'Agree', regex = True)
temp = temp.replace(to_replace= "We have developed a strategy but not implemented it", value = 'Disgree', regex = True)
temp = temp.replace(to_replace= "No Strategy", value = 'Strongly Disagree', regex = True)

df['My organization uses data and insights to inform its corporate strategy.'] = temp

# ----------

temp = df['My organization has an enterprise-wide data and analytics strategy being used.']

temp = df['My organization uses data and insights to inform its corporate strategy.']
temp = temp.replace(to_replace= "No decisions are made automatically with analytics", value = 'Strongly Disagree', regex = True)
temp = temp.replace(to_replace= "Some decisions are made automatically with analytics", value = 'Agree', regex = True)
temp = temp.replace(to_replace= "Most decisions are made automatically with analytics", value = 'Strongly Agree', regex = True)

df['My organization has an enterprise-wide data and analytics strategy being used.'] = temp

# ----------

temp = df['My organization makes automated data-driven decisions.']

temp = temp.replace(to_replace= "Federated - data specialists are both embedded within divisions and in a central team\xa0", value = 'Federated - data specialists are both embedded within divisions and in a central team')
#temp = temp.replace(to_replace= '[No Answer Entered]', value = "Don't Know")

df['My organization makes automated data-driven decisions.'] = temp

# ----------

temp = df['Which structure most closely resembles your organization?*']

temp = temp.replace(to_replace= 'Strongly Agree', value = 'There are people responsible and empowered to make changes')
temp = temp.replace(to_replace= 'Agree', value = 'There are people responsible and empowered to make changes')
temp = temp.replace(to_replace= 'Disagree', value = 'There are people responsible but they are not empowered to make changes')
temp = temp.replace(to_replace= 'Strongly Diagree', value = 'There is no one responsible')
#temp = temp.replace(to_replace= '[No Answer Entered]', value = "Don't Know")

df['Which structure most closely resembles your organization?*'] = temp

# ----------

temp = df['It is easy for everyone in my organization to access data that they are allowed to view.']

temp = temp.replace(to_replace= 'Strongly Agree', value = 'Very Easy')
temp = temp.replace(to_replace= 'Agree', value = 'Easy')
temp = temp.replace(to_replace= 'Disagree', value = 'Difficult')
temp = temp.replace(to_replace= 'Strongly Disagree', value = 'Very Difficult')
temp = temp.replace(to_replace= 'Very difficult', value = 'Very Difficult')
temp = temp.replace(to_replace= 'Very easy', value = 'Very Easy')

df['It is easy for everyone in my organization to access data that they are allowed to view.'] = temp

# ----------

temp = df['My organization outsources the development of digital solutions.']

temp = temp.replace(to_replace= 'Strongly Agree', value = 'All')
temp = temp.replace(to_replace= 'Agree', value = 'Most')
temp = temp.replace(to_replace= 'Disagree', value = 'None')
temp = temp.replace(to_replace= 'Strongly Disagree', value = 'None')
#temp = temp.replace(to_replace= '[No Answer Entered]', value = "Don't Know")

df['My organization outsources the development of digital solutions.'] = temp


In [23]:
temp = df['My organization uses cloud-based tools for most digital solutions.']
temp = temp.replace(to_replace= "No, we don't use cloud", value= 'Not using cloud', regex=True)
use = df['Which technologies does your organization use?']

# putting whatever tech in the cloud column to tech column

for i in range(72, len(temp)):
  if temp[i] == "Not using cloud":
    temp = temp.replace(to_replace= temp[i], value="Strongly Disagree")

  if temp[i] == "AWS, Not using cloud":
    lol = use[i] + ', ' + 'Amazon Web Services'
    use = use.replace(to_replace= use[i], value= lol)
    temp = temp.replace(to_replace= temp[i], value="Disagree")

  if temp[i] == "AWS, Don't know":
    
    lol = use[i] + ', ' + 'Amazon Web Services'
    use = use.replace(to_replace= use[i], value= lol)
    temp = temp.replace(to_replace= temp[i], value= "Don't Know")

  if temp[i] == "Microsoft Azure, Don't know":
    
    lol = use[i] + ', ' + 'Microsoft Azure'
    use = use.replace(to_replace= use[i], value= lol)
    temp = temp.replace(to_replace = temp[i], value="Don't Know")

  if temp[i] == 'start to use snowflake':
    temp = temp.replace(to_replace= temp[i], value="Agree", regex=True)
    lol = use[i]+', '+'Snowflake'
    use = use.replace(to_replace= use[i], value= lol, regex=True)

  if temp[i] == 'Setting up Azure is in progress.':
    lol = use[i]+', '+'Microsoft Azure'
    use = use.replace(to_replace= use[i], value= lol)
    temp = temp.replace(to_replace= temp[i], value="Agree")
  
df['My organization uses cloud-based tools for most digital solutions.'] = temp
df['Which technologies does your organization use?'] = use

In [24]:
temp.unique()

array(['Strongly Disagree', 'Agree', 'Disagree', "Don't Know",
       'Strongly Agree', 'AWS, Microsoft Azure',
       'AWS, Microsoft Azure, Google', 'Microsoft Azure', 'AWS',
       'AWS, Google', "Don't know", 'Google', 'Qliksense Cloud, Google',
       'Microsoft Azure, Google',
       'AWS, Microsoft Azure, Google, Snowflake'], dtype=object)

In [25]:
temp = df['My organization uses cloud-based tools for most digital solutions.']

for i in range(72, len(temp)):
  arr = temp[i].split(',')

  for j in arr:
    if j == 'AWS':
      lol = use[i]+', ' + 'Amazon Web Services'
      use = use.replace(to_replace= use[i], value= lol)
      
    if j == ' Google' or j == 'Google':
      lol = use[i]+', '+'Google Cloud Platform'
      use = use.replace(to_replace= use[i], value= lol)
      
    if j == ' Snowflake' or j == ' Snowflake':
      lol = use[i]+', '+'Snowflake'
      use = use.replace(to_replace= use[i], value= lol, regex=True)
      
    if j == 'Qliksense Cloud':
      lol = use[i]+', '+'Qliksense Cloud'
      use = use.replace(to_replace= use[i], value= lol, regex=True)
    
    if j == ' Microsoft Azure':
      lol = use[i]+', '+'Microsoft Azure'
      use = use.replace(to_replace= use[i], value= lol)
    
    if j == "Don't know" or j == "Don't Know":
      temp = temp.replace(to_replace= temp[i], value= "Don't Know", regex = True)

noList = ['Strongly Disagree', 'Agree', 'Disagree', "Don't Know", 'Strongly Agree']

for i in temp:
  if i not in noList:
    temp = temp.replace(to_replace= i, value= "Strongly Agree")

df['My organization uses cloud-based tools for most digital solutions.'] = temp
df['Which technologies does your organization use?'] = use



---



In [26]:
df['Which technologies does your organization use?'][72]

'Machine learning pipeline, Visualisation tools (e.g. Power BI, Tableau, Qlik etc.), Data warehouse, Data lake, Amazon Web Services, Microsoft Azure, Amazon Web Services, Microsoft Azure, Amazon Web Services, Amazon Web Services'

In [27]:
df.to_csv('Final_Clean_KPMG.csv')